In [ ]:
import pandas as pd
import numpy
from dateutil import parser
from datetime import *
from difflib import SequenceMatcher
from openpyxl import *
from openpyxl.styles import Font, Fill, PatternFill, Alignment
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import ColorScaleRule, CellIsRule, FormulaRule
from openpyxl.worksheet.table import Table, TableStyleInfo

# Reformatting raw CSV data to collect necessary columns
data = pd.read_csv('./JIRA.csv', keep_default_na = False) 

#-------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------
## All columns
# Creating a list out of each column in the data frame
#-------------
#Title
title = data['Summary'].tolist()
entries = len(title)

#Key
key = data['Issue key'].tolist()

#Key TCS
key_tcs = data['Inward issue link (Tyche Client Support Linked Ticket)'].tolist()

#Status
status = data['Status'].tolist()

#Priority
priority = data['Priority'].tolist()

#Resolution
res = data['Resolution'].tolist()

#Assignee
assignee = data['Assignee'].tolist()

#Reporter
reporter = data['Reporter'].tolist()

#--------------------------
# Date Created and Resolved
#--------------------------
date_cre = data['Created'].tolist()
date_res = data['Resolved'].tolist()
# If not resolved, adopt created date, and reformat to 'datetime' format
for i in range(entries):
    if date_res[i] == '':
        date_res[i] = parser.parse(date_cre[i])
    else:
        date_res[i] = parser.parse(date_res[i])
# Created date in 'datetime' format
for i in range(entries):
    date_cre[i] = parser.parse(date_cre[i])

# Age of tickets
age = []
for i in range(entries):
    age.append((datetime.today() - date_cre[i]).days)

# Time to resolve tickets (zero if not resolved)
time_res = list(range(entries))
for i in range(len(time_res)):
    time_res[i] = date_res[i] - date_cre[i]

#-------   
# Client
#-------
client = data['Custom field (Client)'].tolist()
# If empty, populate with 'No Client'
for i in range(entries):
    if client[i] == '':
        client[i] = 'No Client'
# Replace 'Argo (Westfield Specialty)' with 'Argo'
for i in range(entries):
    if client[i] == 'Argo (Westfield Specialty)':
        client[i] = 'Argo'
        
# Define function to fix duplicate client names (temporary, manual)
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
# This function will be used on some clients: Axis, Partner Re. May need to add more if necessary
#'Axis'
for i in range(entries): 
    x = similar(client[i],'Axis')
    if x > 0.8 and x < 1:
        client[i] = 'Axis'
#'Partner Re'
for i in range(entries):
    x = similar(client[i],'Partner Re')
    if x > 0.8 and x < 1:
        client[i] = 'Partner Re'

# Unique Client List - useful list for calculations
client_uni = []
for i in client:
    if i not in client_uni:
        client_uni.append(i)
client_uni = sorted(client_uni)
# Rearrange to make 'No Client' first
client_uni.remove('No Client')
client_uni.insert(0,'No Client')

# Remove redundant clients
if 'Ascot' in client_uni:
    client_uni.remove('Ascot')
if 'TRIUM' in client_uni:
    client_uni.remove('TRIUM')
if 'IT Test' in client_uni:
    client_uni.remove('IT Test')
if 'IT Support Test' in client_uni:
    client_uni.remove('IT Support Test')


#-------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------
## Prep Worksheets
#-----------------
wb = Workbook()
tickets_all = wb.create_sheet('All Tickets')
summary_all = wb.create_sheet('Client Summary')
summary_triage = wb.create_sheet('Triage Summary')
del wb['Sheet']

# Defining important variables
column_list = ['Entry','Ticket Name', 'TPS Key', 'TCS Key', 'Status', 'Priority', 'Resolution', 'Assignee', 'Reporter', 'Created', 'Age', 'Resolved', 'Client']
style_med1 = TableStyleInfo(name='TableStyleMedium1', showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=True)
style_med2 = TableStyleInfo(name='TableStyleMedium2', showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=False)

def set_width():
    ws.column_dimensions['B'].width = 37
    ws.column_dimensions['C'].width = 10
    ws.column_dimensions['D'].width = 10
    ws.column_dimensions['E'].width = 14
    ws.column_dimensions['F'].width = 10
    ws.column_dimensions['G'].width = 13
    ws.column_dimensions['H'].width = 19
    ws.column_dimensions['I'].width = 19
    ws.column_dimensions['J'].width = 20
    ws.column_dimensions['L'].width = 20
    ws.column_dimensions['M'].width = 11
    ws.column_dimensions['O'].width = 12

#-------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------
## All Tickets
#-------------
wb.active = wb['All Tickets']
ws = tickets_all
ws.sheet_properties.tabColor = '00FFFF00'

# Writing table manually
ws.append(column_list)
m=0
for i in range(entries):
    if client[i] != 'Ascot' and client[i] != 'TRIUM' and client[i] != 'IT Test' and client[i] != 'IT Support Test':
        m=m+1
        lst = []
        lst.append(i+1)
        lst.append(title[i])
        lst.append(key[i])
        lst.append(key_tcs[i])
        lst.append(status[i])
        lst.append(priority[i])
        lst.append(res[i])
        lst.append(assignee[i])
        lst.append(reporter[i])
        lst.append(date_cre[i])
        lst.append(age[i])
        if status[i] == 'Closed':
            lst.append(date_res[i])
        else:
            lst.append(' ')
        lst.append(client[i])
        ws.append(lst)
        ws.cell(row=1+m, column=14).value = '=HYPERLINK("{}", "{}")'.format('https://tychesoftware.atlassian.net/browse/' + key[i], 'Go to')
        ws.cell(row=1+m, column=14).style = 'Hyperlink'
        ws.cell(row=1+m, column=15).value = '=HYPERLINK("{}", "{}")'.format('https://tychesoftware.atlassian.net/browse/' + key_tcs[i], 'Go to')
        ws.cell(row=1+m, column=15).style = 'Hyperlink'
    
# Adjust width
set_width()

# Hyperlinks
ws['N1'] = 'Link'
ws['O1'] = 'Link (TCS)'
    
cell_range = 'A1:O' + str(m+1)
tab1 = Table(displayName='Table1', ref=cell_range)
tab1.tableStyleInfo = style_med2
ws.add_table(tab1)

#-------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------
## Client Summary
#----------------
wb.active = wb['Client Summary']
ws = summary_all

ws.append(['Go to Triage Summary', 'Current Period'])
ws.append(['','Outstanding JIRA Tickets (Age in Days)','','','','','','','Avg. Time to Resolve (Days)','','','Outstanding Tickets'])
ws.append(['Client','>0','>10','>15','>21','>28','>30','>60','Avg.','','Priority','Low','Medium','High','Blocker'])
ws.column_dimensions['A'].width = 24
ws.column_dimensions['K'].width = 24
ws['B1'].font = Font(bold=True, underline='single', size=20)
ws['B2'].font = Font(bold=True)
ws['L2'].font = Font(bold=True)

whiteFill = PatternFill(start_color='00FFFFFF', end_color='00FFFFFF', fill_type='solid')
yellowFill = PatternFill(start_color='00FFCC00', end_color='00FFCC00', fill_type='solid')
redFill = PatternFill(start_color='EE1111', end_color='EE1111', fill_type='solid')
# Condition for Age
ws.conditional_formatting.add('B4:H'+str(len(client_uni)+3), CellIsRule(operator='equal', formula=['0'], stopIfTrue=True, fill=whiteFill))
ws.conditional_formatting.add('B4:C'+str(len(client_uni)+3), CellIsRule(operator='greaterThan', formula=['0'], stopIfTrue=True, fill=whiteFill))
ws.conditional_formatting.add('D4:F'+str(len(client_uni)+3), CellIsRule(operator='greaterThan', formula=['0'], stopIfTrue=True, fill=yellowFill))
ws.conditional_formatting.add('G4:H'+str(len(client_uni)+3), CellIsRule(operator='greaterThan', formula=['0'], stopIfTrue=True, fill=redFill))
# Condition for Priority
ws.conditional_formatting.add('L4:O'+str(len(client_uni)+3), CellIsRule(operator='equal', formula=['0'], stopIfTrue=True, fill=whiteFill))
ws.conditional_formatting.add('L4:L'+str(len(client_uni)+3), CellIsRule(operator='greaterThan', formula=['0'], stopIfTrue=True, fill=whiteFill))
ws.conditional_formatting.add('M4:N'+str(len(client_uni)+3), CellIsRule(operator='greaterThan', formula=['0'], stopIfTrue=True, fill=yellowFill))
ws.conditional_formatting.add('O4:O'+str(len(client_uni)+3), CellIsRule(operator='greaterThan', formula=['0'], stopIfTrue=True, fill=redFill))

#-------------------------------------
# Count open tickets by Client and Age
#-------------------------------------
threshold = [0,10,15,21,28,30,60]
threshold_2 = ['Low','Medium','High','Blocker']
lst = []

# Count: No Client only
#----------------------
dummy = ['No Client']
m=0
for k in threshold:
    for i in range(entries):
        if client[i] == 'No Client' and age[i] >= k and res[i] != 'Done':
            m = m+1
    dummy.append(m)
    m=0
n = timedelta(days = 0)
for i in range(entries):
    if time_res[i] > timedelta(days = 0) and client[i] == '':
        m = m+1
        n = n + time_res[i]
if m == 0:
    mean = 'n/a'
else:
    mean = (n.days)/m
dummy.extend([mean,'','No Client'])
m=0
for k in threshold_2:
    for i in range(entries):
        if client[i] == '' and priority[i] == k and res[i] != 'Done':
            m = m+1
    dummy.append(m)
    m=0
lst.append(dummy)

# Count: All other clients
#-------------------------
j=1
while j < len(client_uni):
    dummy = [client_uni[j]]
    m=0
    for k in threshold:
        for i in range(entries):
            if client[i] == client_uni[j] and age[i] >= k and res[i] != 'Done':
                m = m+1
        dummy.append(m)
        m=0
    n = timedelta(days = 0)
    for i in range(entries):
        if time_res[i] > timedelta(days = 0) and client[i] == client_uni[j]:
            m = m+1
            n = n + time_res[i]
    if m == 0:
        mean = 'n/a'
    else:
        mean = (n.days)/m
    dummy.extend([mean,'',client_uni[j]])
    m=0
    for k in threshold_2:
        for i in range(entries):
            if client[i] == client_uni[j] and priority[i] == k and res[i] != 'Done':
                m = m+1
        dummy.append(m)
        m=0
    lst.append(dummy)
    j=j+1         
for row in lst:
    ws.append(row)

# Decimals for average (and align right)
for i in range(len(client_uni)):
    ws['I'+str(i+4)].number_format = '0.00'
    ws['I'+str(i+4)].alignment = Alignment(horizontal='right')
    
#Hyperlink (only for Client <-> Triage)
ws.cell(row=1, column=1).hyperlink = ("#'Triage Summary'!A1")
ws.cell(row=1, column=1).style = 'Hyperlink'

#Hyperlink to each client
for i in range(len(client_uni)):
    address = '#\''+ str(client_uni[i]) +'\'!A1'
    ws.cell(row=4+i, column=1).value = '=HYPERLINK("{}", "{}")'.format(address, client_uni[i])
    ws.cell(row=4+i, column=1).style = 'Hyperlink'
    ws.cell(row=4+i, column=11).value = '=HYPERLINK("{}", "{}")'.format(address, client_uni[i])
    ws.cell(row=4+i, column=11).style = 'Hyperlink'

tab2 = Table(displayName='Table2', ref='A3:I' + str(len(client_uni)+3))
tab21 = Table(displayName='Table2.1', ref='K3:O' + str(len(client_uni)+3))
tab2.tableStyleInfo = style_med1
tab21.tableStyleInfo = style_med1
ws.add_table(tab2)
ws.add_table(tab21)

#-------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------
## Triage Summary
#----------------
wb.active = wb['Triage Summary']
ws = summary_triage

ws.append(['Go to Client Summary', 'Current Period'])
ws.append(['','Outstanding JIRA Tickets in Triage (Age in Days)','','','','','','','','','','Outstanding Tickets in Triage'])
ws.append(['Client','>0','>10','>15','>21','>28','>30','>60','','','Priority','Low','Medium','High','Blocker'])
ws.column_dimensions['A'].width = 24
ws.column_dimensions['K'].width = 24
ws['B1'].font = Font(bold=True, underline='single', size=20)
ws['B2'].font = Font(bold=True)
ws['K2'].font = Font(bold=True)

# (Color)Fill variables defined in Client Summary
# Condition for Age
ws.conditional_formatting.add('B4:H'+str(len(client_uni)+3), CellIsRule(operator='equal', formula=['0'], stopIfTrue=True, fill=whiteFill))
ws.conditional_formatting.add('B4:C'+str(len(client_uni)+3), CellIsRule(operator='greaterThan', formula=['0'], stopIfTrue=True, fill=whiteFill))
ws.conditional_formatting.add('D4:F'+str(len(client_uni)+3), CellIsRule(operator='greaterThan', formula=['0'], stopIfTrue=True, fill=yellowFill))
ws.conditional_formatting.add('G4:H'+str(len(client_uni)+3), CellIsRule(operator='greaterThan', formula=['0'], stopIfTrue=True, fill=redFill))
# Condition for Priority
ws.conditional_formatting.add('L4:O'+str(len(client_uni)+3), CellIsRule(operator='equal', formula=['0'], stopIfTrue=True, fill=whiteFill))
ws.conditional_formatting.add('L4:L'+str(len(client_uni)+3), CellIsRule(operator='greaterThan', formula=['0'], stopIfTrue=True, fill=whiteFill))
ws.conditional_formatting.add('M4:N'+str(len(client_uni)+3), CellIsRule(operator='greaterThan', formula=['0'], stopIfTrue=True, fill=yellowFill))
ws.conditional_formatting.add('O4:O'+str(len(client_uni)+3), CellIsRule(operator='greaterThan', formula=['0'], stopIfTrue=True, fill=redFill))

#-------------------------------------
# Count open tickets by Client and Age
#-------------------------------------

# Count: No Client only
#----------------------
lst = []
dummy = ['No Client']
m=0
for k in threshold:
    for i in range(entries):
        if client[i] == 'No Client' and age[i] >= k and res[i] != 'Done' and status[i] == 'TRIAGE':
            m = m+1
    dummy.append(m)
    m=0
dummy.extend(['','','No Client'])
m=0
for k in threshold_2:
    for i in range(entries):
        if client[i] == '' and priority[i] == k and res[i] != 'Done' and status[i] == 'TRIAGE':
            m = m+1
    dummy.append(m)
    m=0  
lst.append(dummy)

# Count: All other clients
#-------------------------
j=1
while j < len(client_uni):
    dummy = [client_uni[j]]
    m=0
    for k in threshold:
        for i in range(entries):
            if client[i] == client_uni[j] and age[i] >= k and res[i] != 'Done' and status[i] == 'TRIAGE':
                m = m+1
        dummy.append(m)
        m=0
    dummy.extend(['','',client_uni[j]])
    for k in threshold_2:
        for i in range(entries):
            if client[i] == client_uni[j] and priority[i] == k and res[i] != 'Done' and status[i] == 'TRIAGE':
                m = m+1
        dummy.append(m)
        m=0
    lst.append(dummy)
    j=j+1         
for row in lst:
    ws.append(row)

#Hyperlink (only for Client <-> Triage)
ws.cell(row=1, column=1).hyperlink = ("#'Client Summary'!A1")
ws.cell(row=1, column=1).style = 'Hyperlink'

#Hyperlink to each client
for i in range(len(client_uni)):
    address = '#\''+ str(client_uni[i]) +'\'!A1'
    ws.cell(row=4+i, column=1).value = '=HYPERLINK("{}", "{}")'.format(address, client_uni[i])
    ws.cell(row=4+i, column=1).style = 'Hyperlink'
    ws.cell(row=4+i, column=11).value = '=HYPERLINK("{}", "{}")'.format(address, client_uni[i])
    ws.cell(row=4+i, column=11).style = 'Hyperlink'
    
tab3 = Table(displayName='Table3', ref='A3:H' + str(len(client_uni)+3))
tab31 = Table(displayName='Table3.1', ref='K3:O' + str(len(client_uni)+3))
tab3.tableStyleInfo = style_med1
tab31.tableStyleInfo = style_med1
ws.add_table(tab3)
ws.add_table(tab31)

#-------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------
# Tickets list per client
#------------------------
for j in range(len(client_uni)):
    client_tab = wb.create_sheet(client_uni[j])
    ws = client_tab
    wb.active = wb[client_uni[j]]
    ws.sheet_properties.tabColor = '0000FF00'
    ws.append(column_list)
    m=0
    for i in range(entries):
        if client[i] == client_uni[j]:
            m=m+1
            lst = []
            lst.append(i+1)
            lst.append(title[i])
            lst.append(key[i])
            lst.append(key_tcs[i])
            lst.append(status[i])
            lst.append(priority[i])
            lst.append(res[i])
            lst.append(assignee[i])
            lst.append(reporter[i])
            lst.append(date_cre[i])
            lst.append(age[i])
            if status[i] == 'Closed':
                lst.append(date_res[i])
            else:
                lst.append(' ')
            lst.append(client[i])
            ws.append(lst)
            ws.cell(row=1+m, column=14).value = '=HYPERLINK("{}", "{}")'.format('https://tychesoftware.atlassian.net/browse/' + key[i], 'Go to')
            ws.cell(row=1+m, column=14).style = 'Hyperlink'
            ws.cell(row=1+m, column=15).value = '=HYPERLINK("{}", "{}")'.format('https://tychesoftware.atlassian.net/browse/' + key_tcs[i], 'Go to')
            ws.cell(row=1+m, column=15).style = 'Hyperlink'
    ws['N1'] = 'Link'
    ws['O1'] = 'Link (TCS)'
    set_width()
    cell_range = 'A1:O' + str(m+1)
    tab4 = Table(displayName='Table' + str(j+4), ref=cell_range)
    tab4.tableStyleInfo = style_med2
    ws.add_table(tab4)
    
#-------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------
wb.active = wb['All Tickets']
wb.save('./JIRA Auto Report 240311.xlsx')